### Imports
First, make sure the following packages are installed per the [pandas documentation](https://pandas.pydata.org/docs/getting_started/install.html#install-recommended-dependencies):
- `bottleneck`
- `numexpr`

In [1]:
# standard lib
import functools

# third party
import numpy as np
import pandas as pd

In [2]:
df_rows = 1000

df = pd.DataFrame({
    "var1": np.random.randint(1, 100, df_rows),
    "var2": np.random.randint(1, 100, df_rows),
    "var3": np.random.randint(1, 100, df_rows),
    "total": np.random.randint(101, 200, df_rows)
})

In [3]:
df.head()

,var1,var2,var3,total
0,82,13,63,194
1,9,64,84,133
2,90,81,80,133
3,64,16,16,132
4,88,12,51,115


### Base
Base case with just simple `for` loops

In [4]:
%%timeit
sum_list=[]

for i in range(1, 100, 5):
    for j in range(1, 100, 5):
        for k in range(1, 100, 5):
            df_temp = df[(df["var1"] >= i) & (df["var2"] >= j) & (df["var3"] >= k)]
            sum_list.append(df_temp["total"].sum())

print(f"sum_list total is {sum(sum_list)}")

sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
6.44 s ± 159 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Base - no intermediate dataframe
Do not create an intermediate dataframe


In [5]:
%%timeit
sum_list=[]

for i in range(1, 100, 5):
    for j in range(1, 100, 5):
        for k in range(1, 100, 5):
            sum_list.append(df[(df["var1"] >= i) & (df["var2"] >= j) & (df["var3"] >= k)]["total"].sum())

print(f"sum_list total is {sum(sum_list)}")

sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
6.36 s ± 428 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### numpy with `where`
Utilize `where` and numpy

In [6]:
%%timeit
sum_list=[]

for i in range(1, 100, 5):
    for j in range(1, 100, 5):
        for k in range(1, 100, 5):
            sum_list.append(
                np.sum(
                    (np.where(df["var1"] >= i, 1, 0) & np.where(df["var2"] >= j, 1, 0) & np.where(df["var3"] >= k, 1, 0)) * df["total"]
                )
            )

print(f"sum_list total is {sum(sum_list)}")

sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
4.75 s ± 216 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### numpy with `where` and `to_numpy`
Utilize `where` and numpy but convert cols to `numpy` arrays first

In [7]:
%%timeit
sum_list=[]

var1_array = df["var1"].to_numpy()
var2_array = df["var2"].to_numpy()
var3_array = df["var3"].to_numpy()
total_array = df["total"].to_numpy()

for i in range(1, 100, 5):
    for j in range(1, 100, 5):
        for k in range(1, 100, 5):
            sum_list.append(
                np.sum(
                    (np.where(var1_array >= i, 1, 0) & np.where(var2_array >= j, 1, 0) & np.where(var3_array >= k, 1, 0)) * total_array
                )
            )

print(f"sum_list total is {sum(sum_list)}")

sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
284 ms ± 8.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Cache function
Utilize function caching (and partial application)

In [8]:
%%timeit

var1_array = df["var1"].to_numpy()
var2_array = df["var2"].to_numpy()
var3_array = df["var3"].to_numpy()
total_array = df["total"].to_numpy()

def filter_and_mult_total(a1, a2, a3, at, i, j, k):
    return np.sum((np.where(a1 >= i, 1, 0) & np.where(a2 >= j, 1, 0) & np.where(a3 >= k, 1, 0)) * at)

# partial application
# necessary as ndarrays are not hashable
filter_and_mult_total_partial = functools.partial(filter_and_mult_total, var1_array, var2_array, var3_array, total_array)

# cache
@functools.cache
def filter_and_mult_total_cache(i, j, k):
    filter_and_mult_total_partial(i, j, k)

sum_list=[]

for i in range(1, 100, 5):
    for j in range(1, 100, 5):
        for k in range(1, 100, 5):
            sum_list.append(filter_and_mult_total_partial(i, j, k))

print(f"sum_list total is {sum(sum_list)}")

sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
291 ms ± 17.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Cache results of `where` statements
Cache results of `where` statements rather than caching the function call

Basically a form of [dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming) where we save intermediate results and just look them up if they already exist; Python dictionary lookups are fast

In [9]:
%%timeit
sum_list=[]

var1_array = df["var1"].to_numpy()
var2_array = df["var2"].to_numpy()
var3_array = df["var3"].to_numpy()
total_array = df["total"].to_numpy()

i_dict = dict()
j_dict = dict()
k_dict = dict()

for i in range(1, 100, 5):
    # not technically caching but no need to recalculate if i is not changing
    # setdefault returns...
    #   - if key exists, return value
    #   - if key does not exist, add to dictionary, and then return the 2nd argument
    i_result = i_dict.setdefault(i, np.where(var1_array >= i, 1, 0))
    for j in range(1, 100, 5):
        j_result = j_dict.setdefault(j, np.where(var2_array >= j, 1, 0))
        for k in range(1, 100, 5):
            k_result = k_dict.setdefault(k, np.where(var3_array >= k, 1, 0))
            sum_list.append(
                np.sum(
                    (i_result & j_result & k_result) * total_array
                )
            )

print(f"sum_list total is {sum(sum_list)}")

sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 169511126
sum_list total is 16